# Stochastic Ablation Notebook 2
## Conditional circuit branches

This follw-up notebook digs deeper determining conditional probabilities the two heads succeeding or failing together. This is used to map out circuits are branching substructures.

[Run this notebook on Google Colab](https://colab.research.google.com/github/thartlep/stochastic-ablation/blob/main/notebook2.ipynb)

### Some settings to adjust:

In [ ]:
# Nothing yet

### Import Python modules and load data from first notebook:

In [ ]:
# Import needed modules
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import pickle
from collections import defaultdict

# Load data
file_path = "stochastic_ablation__monte_carlo_results.pkl"
with open(file_path, "rb") as f:
    data = pickle.load(f)
results__many_ablated = data["results__many_ablated"]
results__few_ablated = data["results__few_ablated"]
model_name = data["model_name"]
n_layers = data["n_layers"]
n_heads = data["n_heads"]
head_indices = [(l,h) for l in range(n_layers) for h in range(n_heads)]
n_heads_total = len(head_indices)